In [11]:
import gensim
from gensim import corpora
import nltk
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import pandas as pd

[nltk_data] Downloading package stopwords to /home/vscode/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [36]:
df = pd.read_csv('./data/clean_titles.csv')
df.head()

,title,link,date,source,country,language,Manual Sentiment,translated_title,number_of_characters_title,number_of_words_title,...,source_type,final_redirected_URL,domain_of_URL,subdomain_of_URL,URL_depth,top_level_domain,url_length,author,Predicted_Sentiment,cleaned_title
0,Iran uses Artificial Intelligence for maximizi...,https://news.google.com/rss/articles/CBMigAFod...,2023-05-07,ANI News,US,en,-1.0,Iran uses Artificial Intelligence for maximizi...,70,11,...,News Outlet,https://www.aninews.in/news/world/middle-east/...,www.aninews.in,www,4,in,128,[],negative,Iran uses Artificial Intelligence for maximizi...
1,Mortgage Lenders See AI As Key To Operational ...,https://news.google.com/rss/articles/CBMiYGh0d...,2023-10-09,National Mortgage Professional,US,en,1.0,Mortgage Lenders See AI As Key To Operational ...,77,13,...,Press Release Service,https://nationalmortgageprofessional.com/news/...,nationalmortgageprofessional.com,nationalmortgageprofessional,2,com,96,['Christine Stuart'],positive,Mortgage Lenders See AI As Key To Operational ...
2,How the Threat of AI Coincided with the Writer...,https://news.google.com/rss/articles/CBMiX2h0d...,2023-10-04,The Spectator,US,en,-1.0,How the Threat of AI Coincided with the Writer...,59,13,...,News Outlet,https://seattlespectator.com/2023/10/04/how-th...,seattlespectator.com,seattlespectator,5,com,95,"['Hannah Mwangi', 'Volunteer Reporter']",negative,How the Threat of AI Coincided with the Writer...
3,Telex : Le code source du ransomware HelloKitt...,https://news.google.com/rss/articles/CBMiwgFod...,2023-10-09,Le Monde Informatique,US,fr,0.0,Telex: HelloKitty ransomware source code revea...,129,22,...,Other,https://www.lemondeinformatique.fr/actualites/...,www.lemondeinformatique.fr,www,2,fr,194,[],negative,Telex: HelloKitty ransomware source code revea...
4,Is AI a Master or Slave? - Namibian,https://news.google.com/rss/articles/CBMiNGh0d...,2023-10-03,Namibian,US,en,-1.0,Is AI a Master or Slave? - Namibian,28,8,...,Other,https://www.namibian.com.na/is-ai-a-master-or-...,www.namibian.com.na,www,2,na,52,"['Joschka Fischer', 'Please Enter Your Name He...",neutral,Is AI a Master or Slave?


In [33]:

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    tokens = word_tokenize(text)
    # Remove punctuation and numbers
    table = str.maketrans('', '', string.punctuation + string.digits)
    tokens = [word.translate(table) for word in tokens]
    tokens = [token for token in tokens if token not in stop_words and token not in string.punctuation]
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    tokens = [word for word in tokens if len(word) > 2]
    return tokens

In [47]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

processed_headlines = [preprocess_text(headline.lower()) for headline in df['cleaned_title']]

# Create dictionary and corpus
dictionary = corpora.Dictionary(processed_headlines)
corpus = [dictionary.doc2bow(tokens) for tokens in processed_headlines]

# LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word=dictionary, passes=15)

# Print the topics
for idx, topic in lda_model.print_topics():
    print(f'Topic {idx + 1}: {topic}')

dominant_topics = []
# Get the most dominant topic for each headline
for i, headline in enumerate(processed_headlines):
    headline = str(headline)
    bow = dictionary.doc2bow(preprocess_text(headline))
    topic = lda_model.get_document_topics(bow)
    dominant_topic = sorted(topic, key=lambda x: x[1], reverse=True)[0]
    dominant_topics.append(dominant_topic)
    
df['dominant_topic'] = dominant_topics
df.to_csv('data/clean_titles.csv', index=False)

[nltk_data] Downloading package punkt to /home/vscode/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/vscode/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Topic 1: 0.130*"artificial" + 0.127*"intelligence" + 0.006*"tool" + 0.006*"risk" + 0.005*"use" + 0.005*"new" + 0.005*"expert" + 0.005*"world" + 0.005*"impact" + 0.005*"develop"
Topic 2: 0.016*"new" + 0.009*"next" + 0.008*"call" + 0.008*"voice" + 0.007*"company" + 0.007*"regulation" + 0.006*"generative" + 0.006*"robot" + 0.005*"first" + 0.005*"unveils"
Topic 3: 0.021*"musk" + 0.019*"chatbot" + 0.016*"elon" + 0.013*"say" + 0.010*"grok" + 0.008*"use" + 0.008*"like" + 0.008*"human" + 0.007*"microsoft" + 0.006*"robot"
Topic 4: 0.017*"generative" + 0.011*"law" + 0.008*"content" + 0.008*"free" + 0.008*"tool" + 0.007*"chatgpt" + 0.006*"new" + 0.006*"ceo" + 0.006*"claude" + 0.006*"big"
Topic 5: 0.042*"gpt" + 0.024*"chat" + 0.016*"google" + 0.013*"chatbot" + 0.012*"model" + 0.012*"chatgpt" + 0.010*"health" + 0.008*"bard" + 0.008*"generative" + 0.008*"openai"
Topic 6: 0.029*"model" + 0.029*"generative" + 0.018*"language" + 0.018*"chatbot" + 0.017*"google" + 0.014*"launch" + 0.011*"new" + 0.011*"c

'''
Topic 0: 0.013*"generative" + 0.010*"new" + 0.007*"help" + 0.007*"launch" + 0.005*"tool" + 0.005*"aipowered" + 0.005*"make" + 0.005*"enterprise" + 0.005*"chatbots" + 0.005*"platform"
Topic 1: 0.060*"artificial" + 0.058*"intelligence" + 0.009*"generative" + 0.009*"chatbot" + 0.007*"musk" + 0.006*"human" + 0.006*"use" + 0.005*"risk" + 0.005*"elon" + 0.005*"say"
Topic 2: 0.031*"artificial" + 0.030*"intelligence" + 0.011*"new" + 0.010*"chatbot" + 0.010*"generative" + 0.010*"stock" + 0.009*"gpt" + 0.007*"launch" + 0.007*"chat" + 0.007*"expert"
Topic 3: 0.022*"model" + 0.012*"gpt" + 0.010*"new" + 0.009*"chatbot" + 0.009*"language" + 0.008*"open" + 0.008*"llm" + 0.007*"generative" + 0.007*"large" + 0.006*"market"
Topic 4: 0.025*"generative" + 0.012*"google" + 0.011*"meta" + 0.011*"new" + 0.010*"intelligence" + 0.009*"chatbot" + 0.009*"artificial" + 0.008*"tool" + 0.007*"launch" + 0.007*"model"
'''